In [17]:
import csv as csv 
import numpy as np
csv_file_object = csv.reader(open('c:Desktop/titanic/train.csv', 'rt'))
header = next(csv_file_object)

data=[]
for row in csv_file_object:
    data.append(row)
data = np.array(data)   
#print(data)
#print(data[0])
#print(data[0,3])
#print(data[0:,4])
number_passengers = np.size(data[0:,1].astype(np.float))
#print(number_passengers)
number_survived = np.sum(data[0:,1].astype(float))
#print(number_survived)
proportion_survivors = number_survived / number_passengers
# the two below will show all the male and the female in separately
women_only_stats = data[0:,4] == "female"
men_only_stats = data[0:,4] != "female"

#women onboard
women_onboard = data[women_only_stats, 1].astype(float)
men_onboard = data[men_only_stats, 1].astype(float)

#proportions of them that survived
proportion_women_survived = np.sum(women_onboard)/np.size(women_onboard)

#proportion of the men that survived
proportion_men_survived = np.sum(men_onboard)/np.size(men_onboard)

print(proportion_women_survived)
print(proportion_men_survived)

#reading the test data and wrriting the gender model as a csv
test_file = open('C:Desktop/titanic/test.csv', 'rt')
test_file_object = csv.reader(test_file)
header = next(test_file_object)   

#creating a new file that we shall point to when dropping in the data
prediction_file = open("genderbasedmodel.csv", 'wt')
prediction_file_object = csv.writer(prediction_file) 

#read the test file , see if it is male or female and write our survival predic
#into a new file basing on the fact that a woman survived and a make survived

prediction_file_object.writerow(['PassengerID', 'Survived'])                               
for row in test_file_object:
    if row[3] == 'female':
       prediction_file_object.writerow([row[0], '1']) 
    else:
        prediction_file_object.writerow([row[0], '0'])                       
test_file.close()
prediction_file.close()                               
                        

                               
                                                               
    
            
                               
                               
                               
                               

                               

0.742038216561
0.188908145581


In [3]:
#now we create a model which uses the price, class and gender
#author ronald turyatemba

import csv
import numpy as np
csv_file_object = csv.reader(open('C:Desktop/titanic/train.csv', 'rt'))

#skip the first line as it is a header
header = next(csv_file_object)

#create a variable to hold the data
data=[]

for row in csv_file_object:
    data.append(row)
#i changed the list data into an array to use numerical py on    
data = np.array(data)    

# In order to analyse the price column I need to bin up that data
# here are my binning parameters, the problem we face is some of the fares are very large
# So we can either have a lot of bins with nothing in them or we can just lose some
# information by just considering that anythng over 39 is simply in the last bin.
# So we add a ceiling
fare_ceiling = 40

#modify the data in the fare column to = 39 if greater or equal the ceiling
data[data[0:,9].astype(float) >= fare_ceiling, 9] = fare_ceiling - 1.0

fare_bracket_size = 10
#i am using the // operato that is the floor divisor
#ie dividing will bring a float and the range method doesnt take in floats so i 
#changed it to // which will take the floor of the result from dividing that will be an integer
number_of_price_brackets = (fare_ceiling // fare_bracket_size)
#number_of_classes = 3 #there are there classses 1st 2nd and 3rd
#but its better to use from the pclass directly because there before we knew the number

number_of_classes = len(np.unique(data[0::,2]))
#print(number_of_classes)

#we creating a table that will have all the data and we initialize this table tozeros all the three parameters in the list

survival_table = np.zeros([2,number_of_classes,number_of_price_brackets], float)
# we are using xrange because it will generate on interger at a time unlike range which
# will generate al the value at once and there chew up your memory e.g. range(5) will bring up 
#[0,1,2,3,4] wil xrange will bring the same but one by one at run time
#python 3 changes that to range from xrange
for i in range(number_of_classes):
    for j in range(number_of_price_brackets):
        women_only_stats = data[(data[0:,4]=="female") \
                                 &(data[0:,2].astype(float) == i+1) \
                                 &(data[0:,9].astype(float) >= j*fare_bracket_size) \
                                 &(data[0:,9].astype(float) < (j+1)*fare_bracket_size) \
                                , 1]
        men_only_stats = data[(data[0:,4]!="female")
                             &(data[0:,2].astype(float) == i+1) \
                                 &(data[0:,9].astype(float) >= j*fare_bracket_size) \
                                 &(data[0:,9].astype(float) < (j+1) *fare_bracket_size ) \
                                 , 1]
#stats for women
        survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float))  
#stats for men         
        survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))
# since in python if it tries to find the mean of an array without anything in it
# (such that the denominator is 0), then it returns nan, we can convert these to 0
# # by just saying where does the array not equal the array, and set these to 0.  
survival_table[ survival_table != survival_table ] = 0

# Now I have my proportion of survivors, simply round them such that if <0.5
# I predict they dont surivive, and if >= 0.5 they do
print(survival_table)
survival_table[ survival_table < 0.5 ] = 0
survival_table[ survival_table >= 0.5 ] = 1
print(survival_table)

#reading test
test_file = open ("C:Desktop/titanic/test.csv", 'rt')
test_file_object = csv.reader(test_file)
header = next(test_file_object)

predictions_file = open("genderclassmodel", 'wt')
predictions_file_object = csv.writer(predictions_file)
predictions_file_object.writerow(["PassengerId", "Survived"])

#First lets bin up the fares on the test file
for row in test_file_object:
        for j in range(number_of_price_brackets):
        #if there is no fare then place the price of the ticket according to class
            try:
                row[8] = float(row[8]) #check if the content of the row on column 8 has a value, if it isnt an exception will be thrown as defined
            except:
                  #we put them in the first bin_fare with an index 0 ie the first group 0-9
                bin_fare = number_of_price_brackets-1
            # then then break out of the loop and go to the next row
            break 
        
            #if we have something greater than the ceiling value
            if row[8] > fare_ceiling:
                bin_fare = number_of_price_brackets-1 #this will put that passenger in bin_fare with index 3 ie the highest
                break
            #if the fare is between 0 and 9 inclusive    
            if (row[8] >= j*fare_bracket_size and row[8])< ((j+1)*fare_bracket_size):
                bin_fare = j
                break
         
        #now that we have binned fare, lets cross ref their details with our survival tabel
        if row[3] == "female":
            predictions_file_object.writerow([row[0], "%d" %int(survival_table[0, float(row[1]) -1, bin_fare])])
        
        else:
            predictions_file_object.writerow([row[0], "%d" %int(survival_table[1, float(row[1]) -1, bin_fare])])
        
#close the files then
test_file.close()
predictions_file.close()
print(survival_table)
        
        
        
        


3
[[[ 0.          0.          0.83333333  0.97727273]
  [ 0.          0.91428571  0.9         1.        ]
  [ 0.59375     0.58139535  0.33333333  0.125     ]]

 [[ 0.          0.          0.4         0.38372093]
  [ 0.          0.15873016  0.16        0.21428571]
  [ 0.11153846  0.23684211  0.125       0.24      ]]]
[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


c:\users\admin\appdata\local\programs\python\python35-32\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
c:\users\admin\appdata\local\programs\python\python35-32\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'bin_fare' is not defined